In [ ]:
### REMOVE LATER ###
# Go up one directory since notebook inside task3/ folder.
import os
try:
    if UP_DIR:
        print("skipping")
except NameError:
    os.chdir("..")
    UP_DIR = True

# JH's Task 3 Experiment 1
(NOTE: keep this H1 header block or add it later to denote the boundaries between notebooks when we combined later)

I don't know what I am doing yet for this.

## Imports

In [ ]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import xgboost as xgb

## Hyperparameters

In [ ]:
@dataclass
class Hparams:
    val_split: float = 0.2
    seed: int = 42
    
    pca_n_components = 400
    

HP = Hparams()

In [ ]:
TRAIN_CSV = "./data/train.csv"
TRAIN_TFIDF_CSV = "./data/train_tfidf_features.csv"
TEST_CSV = "./data/test.csv"
TEST_TFIDF_CSV = "./data/test_tfidf_features.csv"

## Data Engineering
Instead of using the tfidf features given in the comp, we can engineer our own tfidf features with better filtering logic, or using something other than tfidf altogether to arrive at vector representations, or perhaps even use a strategy that use non-vector representations.

### Load Data

In [ ]:
train_df = pd.read_csv(TRAIN_CSV, index_col="id")
train_tfidf_df = pd.read_csv(TRAIN_TFIDF_CSV, index_col="id")
test_df = pd.read_csv(TEST_CSV, index_col="id")
test_tfidf_df = pd.read_csv(TEST_TFIDF_CSV, index_col="id")

In [ ]:
def tfidf_to_np(df: pd.DataFrame):
    """Convert the tfidf CSVs to X array of features and y array of labels, ordered
    by id.
    """
    df = df.sort_index()

    if "label" in df.columns:
        y = df.pop("label").to_numpy()
    else:
        y = None

    X = df.to_numpy()
    return X, y

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(*tfidf_to_np(train_tfidf_df), test_size=HP.val_split, random_state=HP.seed)
test_X, test_y = tfidf_to_np(test_tfidf_df)

print("Dataset splits:")
print("  train:", len(train_X))
print("  val:  ", len(val_X))
print("  test: ", len(test_X))

### Fit PCA For Dim Reduction

In [ ]:
model_pca = PCA(n_components=HP.pca_n_components, random_state=HP.seed)

model_pca.fit(train_X)

In [ ]:
ratios = list(zip(range(HP.pca_n_components), model_pca.explained_variance_ratio_))
ratios.sort(key=lambda x: x[1], reverse=True)

print("Most Informative Dimensions:")
for idx, val in ratios[:3]:
    print(f"  Dim {idx}: {val}")

print("Noise:", model_pca.noise_variance_)
print("Total Explained Variance:", sum(model_pca.explained_variance_))

In [ ]:
t_train_X = model_pca.transform(train_X)
t_val_X = model_pca.transform(val_X)
t_test_X = model_pca.transform(test_X)

## Fit XGBoost
Its boosted trees or random forests for both classification or regression.

Tutorial: <https://xgboost.readthedocs.io/en/stable/python/python_intro.html#setting-parameters>

In [ ]:
dtrain = xgb.DMatrix(t_train_X, label=train_y)
dval = xgb.DMatrix(t_val_X, label=val_y)

In [ ]:
# https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html
params = {
    "validate_parameters": True,
    
    "colsample_bynode": 0.8,
    "learning_rate": 0.25,
    "max_depth": 4,
    "num_parallel_tree": 500,
    # TODO: when you use hinge loss cuz its easier to understand but actually logistic
    # regression converges faster, but actually you should just add another eval metric
    # "objective": "binary:hinge",
    "objective": "binary:logistic",
    "subsample": 0.6,
    "tree_method": "hist",
    "device": "gpu",
}
num_round = 200
early_stopping_rounds = 10
# Last set is used by xgb's early stopping.
eval_list = [(dtrain, "train"), (dval, "val")]

In [ ]:
results = {}
bst = xgb.train(
    params,
    dtrain,
    num_round,
    evals=eval_list,
    evals_result=results,
    early_stopping_rounds=early_stopping_rounds,
)

In [ ]:
#xgb.plot_importance(bst)

## Inference

In [ ]:
dtest = xgb.DMatrix(t_test_X)

pred_y = bst.predict(dtest, iteration_range=(0, bst.best_iteration+1))

In [ ]:
pred_df = pd.DataFrame(
    zip(test_tfidf_df.index, np.where(pred_y > 0.5, 1, 0)), columns=["id", "label"]
)
pred_df.to_csv("submission.csv", index=False)